## AgentTorch API

- Execute simulations
- Customize agents
- Customize populations
- Engage with results

In [1]:
import pandas as pd
import os

#### Simulation Input

`Simulation Input` : Input to our simulation is a dataset consisting of population generated based on the census data of a specific region. We have currently generated population for New York City and New Zealand.


Lets see how the population data looks like for New Zealand

In [3]:
NZ_SYNTHETIC_POPULATION_PATH = os.path.join(os.getcwd(), 'simulator_data/census_populations/NZ/synthetic_populations/population_data.pkl')

nz_population = pd.read_pickle(NZ_SYNTHETIC_POPULATION_PATH)
nz_population.head(5)

,area,age,gender,ethnicity,region
0,204200,U19,female,Maori,Bay of Plenty
1,204200,U19,female,Maori,Bay of Plenty
2,204200,U19,male,European,Bay of Plenty
3,204200,U19,female,Maori,Bay of Plenty
4,204200,U19,female,Maori,Bay of Plenty


The above data shows the population attributes of 5 agents, like 'age', 'gender', 'region where the agent resides', 'ethnicity' and 'area code of residence'. We assign these attributes to each of our simulated agents, to mimic real world population dynamic.

We also provide APIs to generate your own synthetic population, though we will not get into the details here.

## Run Simulation

`Dataloader` is the common interface to load, setup or modify the simulation parameters such as Number of Runs, Population Size, Region for which Population data is to be used. It also does the necessary pipelining to setup the flow of simulation.

It compiles a config file which is used by the Executor to run the simulation.

In [22]:
from AgentTorch.dataloader import DataLoader

# 'populations' is the package where generated population for each region is saved.
# We are picking New Zealand's (NZ) population for this run.
from populations import NZ

# 'models' is the package where the simulation logic for different use cases are defined.
# We are picking the macro_economics model for this simulation. It models the population's economic behavior during covid, by simulating a financial market.
from models import macro_economics
from macro_economics.config import NUM_AGENTS

# Initialize the data loader with the preferred model and the region's population.
data_loader = DataLoader(model = macro_economics, region = NZ, population_size = NUM_AGENTS)


Config saved at:  /Users/shashankkumar/Documents/GitHub/MacroEcon/models/macro_economics/yamls/config.yaml
Config saved at:  /Users/shashankkumar/Documents/GitHub/MacroEcon/models/macro_economics/yamls/config.yaml
Config saved at:  /Users/shashankkumar/Documents/GitHub/MacroEcon/models/macro_economics/yamls/config.yaml


Example 1: Use `Executor` API to run a simulation

In [5]:
# Executor is the common interface to run simulation
# It takes the model and data loader as input and executes the simulation.
from AgentTorch.execute import Executor

agents_sim = Executor(model = macro_economics,data_loader = data_loader)
agents_sim.execute()



running episode 0...
Substep Action: Earning decision
LLM benchmark expts
Some Example Prompts:  ['You are female of age U19, living in the Bay of Plenty region. It is September 2020, number of covid cases is 874.0. The price of Essential Goods is 12.0. With all these factors in play, and considering aspects like your living costs, any future aspirations, and the broader economic trends, how is your willingness to work this month? Furthermore, how would you plan your expenditures? ', 'You are female of age U19, living in the Otago region. It is September 2020, number of covid cases is 874.0. The price of Essential Goods is 12.0. With all these factors in play, and considering aspects like your living costs, any future aspirations, and the broader economic trends, how is your willingness to work this month? Furthermore, how would you plan your expenditures? ', 'You are female of age U19, living in the Tasman region. It is September 2020, number of covid cases is 874.0. The price of Ess

Example 2: Use `DataLoader` API to customize the agent - Make LLM an Agent

Let's modify the prompt used by our LLM Agent, and re-run the simulation

Original Prompt is : "You are {gender} of age {age}, living in the {region} region. It is {month} {year}, number of covid cases is {covid_cases}. The price of Essential Goods is {price_of_goods}. With all these factors in play, and considering aspects like your living costs, any future aspirations, and the broader economic trends, how is your willingness to work this month? Furthermore, how would you plan your expenditures? "

We will modify this to remove the information about `price of essential goods`. This might, help us understand the population/market behaviour when info on inflation is not present.

In [23]:
MODIFIED_EARNING_ACTION_PROMPT = "You are {gender} of age {age}, living in the {region} region. It is {month} {year}, number of covid cases is {covid_cases}. With all these factors in play, and considering aspects like your living costs, any future aspirations, and the broader economic trends, how is your willingness to work this month? Furthermore, how would you plan your expenditures? "
data_loader.set_config_attribute('EARNING_ACTION_PROMPT', MODIFIED_EARNING_ACTION_PROMPT)

agent_sim_modified = Executor(model = macro_economics,data_loader = data_loader)
# With this set, we can now run the simulation again using the executor.
agent_sim_modified.execute()

Config saved at:  /Users/shashankkumar/Documents/GitHub/MacroEcon/models/macro_economics/yamls/config.yaml
resolvers already registered..

running episode 0...
Substep Action: Earning decision
LLM benchmark expts
Some Example Prompts:  ['You are female of age U19, living in the Bay of Plenty region. It is September 2020, number of covid cases is 874.0. With all these factors in play, and considering aspects like your living costs, any future aspirations, and the broader economic trends, how is your willingness to work this month? Furthermore, how would you plan your expenditures? ', 'You are female of age U19, living in the Otago region. It is September 2020, number of covid cases is 874.0. With all these factors in play, and considering aspects like your living costs, any future aspirations, and the broader economic trends, how is your willingness to work this month? Furthermore, how would you plan your expenditures? ', 'You are female of age U19, living in the Tasman region. It is Se

## Visualize Results

Simulations produce a state trace
 
 More info on state trace ....

In [9]:
from macro_economics.config import STATE_TRACE_PATH
from AgentTorch.LLM.llm_qa import load_state_trace

state_trace = load_state_trace(STATE_TRACE_PATH,NZ_NUM_AGENTS)
state_trace[0].columns

Index(['ID', 'age', 'area', 'assets', 'consumption_propensity', 'ethnicity',
       'gender', 'household', 'monthly_income', 'post_tax_income', 'region',
       'will_work', 'month', 'year'],
      dtype='object')

Example 3: Use `Visualizer` API to engage the simulation trace for retrospective and prospective analysis 

In [10]:
# Visualizer is the helper class to visualize the output variables of the simulation
# As we would expect, it takes simulation trace as input
from AgentTorch.visualize import Visualizer
visualizer = Visualizer(state_trace)

In [11]:
# We can plot a few variables of interest like Consumption Propensity and Will to Work.
visualizer.plot('consumption_propensity')

In [12]:
# Next lets visualize the Aggregate Willingness to work in each region
visualizer.plot('will_to_work')

## Talk with Simulation

`Q/A` : AgentTorch gives you the ability to perform Q/A with the simulation, through its integration of simulation trace and conversation history with th Analysis Agent (LLM Agent).

##### Configuration

In [13]:
from macro_economics.config import OPENAI_API_KEY
conversations_memory_directory = ROOT_DIR + 'populations/NZ/simulation_memory_output/2020/9'
state_trace_path = ROOT_DIR + 'populations/NZ/state_data_dict.pkl'

##### Analyze

`DocumentRetriever` : It sets up a vector db containing all the converations intiated with the LLM Agent, during the runtime of the simulation.

In [15]:
from AgentTorch.LLM.llm_qa import DocumentRetriever
conversation_memory_retriever = DocumentRetriever(directory=conversation_memory_directory)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

`SimulationAnalysisAgent` abstracts away the logic for analysis module, so you can do what's more fun, that is analyzing the simulation behaviour. 

In [16]:
from AgentTorch.LLM.llm_qa import SimulationAnalysisAgent
analyser = SimulationAnalysisAgent(openai_api_key=OPENAI_API_KEY, document_retriever=memory_retriever, temperature=0, state_trace_path=state_trace_path)

In [17]:
analyser.query("Which age group has lowest median income, how much is it?")



> Entering new AgentExecutor chain...

Invoking: `run_analysis_on_simulation_state` with `{'query': 'Which age group has the lowest median income and how much is it?'}`


The age group with the lowest median income is 20t29 with a median income of $168.00.The age group with the lowest median income is 20-29 years old, with a median income of $168.00.

> Finished chain.


{'input': 'Which age group has lowest median income, how much is it?',
 'output': 'The age group with the lowest median income is 20-29 years old, with a median income of $168.00.'}

In [18]:
analyser.query("Which age group has highest consumption")



> Entering new AgentExecutor chain...

Invoking: `run_analysis_on_simulation_state` with `{'query': 'Which age group has the highest consumption?'}`


The age group with the highest consumption is 65A.The age group with the highest consumption is 65 and above.

> Finished chain.


{'input': 'Which age group has highest consumption',
 'output': 'The age group with the highest consumption is 65 and above.'}

In [20]:
analyser.query("How is inflation affecting consumption behaviour?")



> Entering new AgentExecutor chain...

Invoking: `simulation_memory_retriever` with `{'query': 'inflation and consumption behaviour'}`




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Reasoning: Let's think step by step in order to produce the answer. We observe a decrease in the number of COVID cases from 874.0 to 701.0, indicating a potential improvement in the pandemic situation in the Wellington region. This improvement might lead to a higher willingness to work due to perceived lower health risks and possibly more job opportunities as the economy might start to recover. The slight increase in the price of Essential Goods from 12.0 to 12.012574195861816 suggests a minimal inflation in the cost of living, which might not significantly impact the overall expenditure strategy but indicates economic activity. Given the age group (30t39), there's likely a strong incentive to maintain or increase income levels to support any current or future financial goals, such as savings, investments,

Reasoning: Let's think step by step in order to produce the answer. Given the decrease in the number of COVID cases from 874.0 to 701.0, there is a slight improvement in the pandemi

{'input': 'How is inflation affecting consumption behaviour?',
 'output': 'The analysis of the simulation data shows that inflation has a varying impact on consumption behavior across different demographics. Here are some key observations:\n\n1. For younger age groups (20-29), the impact of inflation on consumption propensity is minimal. This could be due to their relatively lower assets and income levels.\n\n2. For the age group under 19, the impact of inflation is more pronounced. This could be due to their reliance on fixed income sources like allowances or part-time jobs, which do not adjust for inflation.\n\n3. Across different regions, the impact of inflation on consumption behavior also varies. For instance, in the Wellington region, the impact of inflation is more noticeable compared to the Bay of Plenty region.\n\n4. The impact of inflation on consumption behavior also varies by ethnicity and gender. For instance, the Asian and Hispanic communities show a higher impact of infl

#### Bring your own Population

In [21]:
from models import macro_economics
from populations import NYC
from macro_economics.config import NYC_NUM_AGENTS

data_loader_nyc = DataLoader(model = macro_economics, region = NYC, population_size = NYC_NUM_AGENTS)
agents_sim = Executor(macro_economics,data_loader_nyc)
agents_sim.execute()

Config saved at:  /Users/shashankkumar/Documents/GitHub/MacroEcon/models/macro_economics/yamls/config.yaml
Config saved at:  /Users/shashankkumar/Documents/GitHub/MacroEcon/models/macro_economics/yamls/config.yaml
Config saved at:  /Users/shashankkumar/Documents/GitHub/MacroEcon/models/macro_economics/yamls/config.yaml
resolvers already registered..

running episode 0...
Substep Action: Earning decision
LLM benchmark expts
Some Example Prompts:  ['You are male of age 20t29, living in the BK region. It is September 2020, number of covid cases is 874.0. With all these factors in play, and considering aspects like your living costs, any future aspirations, and the broader economic trends, how is your willingness to work this month? Furthermore, how would you plan your expenditures? ', 'You are male of age 20t29, living in the BX region. It is September 2020, number of covid cases is 874.0. With all these factors in play, and considering aspects like your living costs, any future aspiratio